In [1]:
from google.colab import drive
drive.mount('/content/drive')

!nvidia-smi

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Fri Aug 13 17:19:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+------

In [2]:
BASE_PATH = "drive/MyDrive/p2p-torch/"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir
!pip install wandb --upgrade

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-raia4rh7
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-raia4rh7


In [5]:
import wandb
wandb.login()

wandb: Currently logged in as: jrchang (use `wandb login --relogin` to force relogin)


True

In [38]:
!python drive/MyDrive/p2p-torch/train.py

wandb: Currently logged in as: jrchang (use `wandb login --relogin` to force relogin)
2021-08-13 17:08:45.821535: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
wandb: Tracking run with wandb version 0.12.0
wandb: Syncing run spring-music-13
wandb: ⭐️ View project at https://wandb.ai/jrchang/pix2pix
wandb: 🚀 View run at https://wandb.ai/jrchang/pix2pix/runs/1qyjcg1f
wandb: Run data is saved locally in /content/wandb/run-20210813_170844-1qyjcg1f
wandb: Run `wandb offline` to turn off syncing.

Epoch: 0 

100% 4/4 [00:06<00:00,  1.62s/it, D_fake=0.517, D_real=0.499]
=> Saving checkpoint
drive/MyDrive/p2p-torch/checkpoints/gen.pt
=> Saving checkpoint
drive/MyDrive/p2p-torch/checkpoints/disc.pth.tar
Epoch: 1 

100% 4/4 [00:02<00:00,  1.47it/s, D_fake=0.48, D_real=0.504]
Epoch: 2 

100% 4/4 [00:02<00:00,  1.47it/s, D_fake=0.444, D_real=0.535]
Epoch: 3 

100% 4/4 [00:02<00:00,  1.48it/s, D_fake=0.409, D_real=0.604]
=> Sav

KeyboardInterrupt: ignored

In [7]:
!ln -s drive/MyDrive/p2p-torch/generator_model.py generator_model.py
!ln -s drive/MyDrive/p2p-torch/config.py config.py

ln: failed to create symbolic link 'generator_model.py': File exists
ln: failed to create symbolic link 'config.py': File exists


## Helpers

In [8]:
import os
import time
import config
import torch
import torchvision
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
from torch.utils.mobile_optimizer import optimize_for_mobile

class MapDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.list_files = os.listdir(self.root_dir)

    def __len__(self):
        return len(self.list_files)

    def __getitem__(self, index):
        img_file = self.list_files[index]
        img_path = os.path.join(self.root_dir, img_file)
        image = np.array(Image.open(img_path))
        input_image = image[:, :512, :]
        target_image = image[:, 512:, :]

        augmentations = config.both_transform(image=input_image, image0=target_image)
        input_image = augmentations["image"]
        target_image = augmentations["image0"]

        input_image = config.transform_only_input(image=input_image)["image"]
        target_image = config.transform_only_mask(image=target_image)["image"]

        return input_image, target_image

def print_model_size(mdl):
    torch.save(mdl.state_dict(), "tmp.pt")
    print("Model Size: %.2f MB" %(os.path.getsize("tmp.pt")/1e6))
    os.remove('tmp.pt')

def model_speedrun(M):
  M.eval()
  dataset = MapDataset(BASE_PATH + "cherries/val/")
  loader = DataLoader(dataset, batch_size=1)
  for x, y in loader:
      save_image(x * 0.5 + 0.5, BASE_PATH + "in.png")
      s = time.time()
      x = M(x)
      elapsed = time.time() - s
      x = x * 0.5 + 0.5
      save_image(x, BASE_PATH + "out.png")
      print('''\nelapsed time (seconds): {0:.1f}'''.format(elapsed))
      print("Saved Input and Output Image to " + BASE_PATH)
      break

def model_trace(M, name="pix2pix_traced"):
  M.eval()
  example = torch.rand(1, 3, 512, 512)
  traced_script_module = torch.jit.trace(M, example)
  traced_script_module_optimized = optimize_for_mobile(traced_script_module)
  traced_script_module_optimized._save_for_lite_interpreter(f"{BASE_PATH}checkpoints/{name}.pt")



## Load Benchmark

In [10]:
generator_path = BASE_PATH + 'checkpoints/pix2pix_traced_josh.pt'
model = torch.jit.load(generator_path, map_location="cpu")
print(generator_path)
print_model_size(model)

model_speedrun(model)

drive/MyDrive/p2p-torch/checkpoints/pix2pix_traced_josh.pt
Model Size: 0.00 MB

elapsed time (seconds): 0.2
Saved Input and Output Image to drive/MyDrive/p2p-torch/


In [44]:
model_trace(model, 'pix2pix_traced_josh')

## Quantization

### Workflow: POST TRAINING DYNAMIC QUANTIZATION (Not ideal for models with Conv Layers)

In [ ]:
qmodel_dynamic = torch.quantization.quantize_dynamic(model, dtype=torch.quint8)

print_model_size(qmodel_dynamic)

Model Size: 217.72 MB


### Workflow: POST TRAINING STATIC QUANTIZATION

### Step 1

In [ ]:
model.eval()
layers_to_fuse = [
  ["down1.conv.0", "down1.conv.1"], 
  ["down2.conv.0", "down2.conv.1"], 
  ["down3.conv.0", "down3.conv.1"], 
  ["down4.conv.0", "down4.conv.1"], 
  ["down5.conv.0", "down5.conv.1"], 
  ["down6.conv.0", "down6.conv.1"], 
  ["bottleneck.0", "bottleneck.1"],
  ["up1.conv.1", "up1.conv.2"],
  ["up2.conv.1", "up2.conv.2"],
  ["up3.conv.1", "up3.conv.2"],
  ["up4.conv.1", "up4.conv.2"],
  ["up5.conv.1", "up5.conv.2"],
  ["up6.conv.1", "up6.conv.2"],
  ["up7.conv.1", "up7.conv.2"],
]
qmodel_static = torch.quantization.fuse_modules(model, layers_to_fuse)

print_model_size(qmodel_static)

Model Size: 217.69 MB


In [ ]:
qmodel_static.modules

<bound method Module.modules of Generator(
  (initial_down): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (down1): Block(
    (conv): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
      (1): Identity()
      (2): LeakyReLU(negative_slope=0.2)
    )
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (down2): Block(
    (conv): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
      (1): Identity()
      (2): LeakyReLU(negative_slope=0.2)
    )
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (down3): Block(
    (conv): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
      (1): Identity()
      (2): LeakyReLU(negative_slope=0.2)
    )
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (dow

### Step 2. Prepare and Calibrate

In [ ]:
qmodel_static = torch.quantization.prepare(model, inplace=False)
qmodel_static.qconfig = torch.quantization.get_default_qconfig('qnnpack')

qmodel_static.eval()

dataset = MapDataset(BASE_PATH + "cherries/train/")
data_loader = DataLoader(dataset, batch_size=16)

for batch, target in data_loader:
  model(batch)

torch.quantization.convert(qmodel_static)
print(qmodel_static.down1)

/usr/local/lib/python3.7/dist-packages/torch/quantization/quantize.py:244: UserWarning: None of the submodule got qconfig applied. Make sure you passed correct configuration through `qconfig_dict` or by assigning the `.qconfig` attribute directly on submodules
  warnings.warn("None of the submodule got qconfig applied. Make sure you "


Block(
  (conv): Sequential(
    (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False, padding_mode=reflect)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
  )
  (dropout): Dropout(p=0.5, inplace=False)
)


In [ ]:
model_speedrun(qmodel_static)
print_model_size(qmodel_static)

Saved Input and Output Image to drive/MyDrive/p2p-torch/
Model Size: 217.72 MB


### Prototype Feature: GRAPH MODE QUANTIZATION


### Trial 1

In [ ]:
generator_path = BASE_PATH + 'checkpoints/p2p.pt'
model = torch.load(generator_path, map_location="cpu")

scripted_model = torch.jit.script(model)

model_prototype = torch.quantization.prepare_jit(scripted_model, {"": torch.quantization.default_qconfig})

model_prototype = torch.quantization.convert_jit(model_prototype)

print_model_size(model_prototype)
model_speedrun(model_prototype)

/usr/local/lib/python3.7/dist-packages/torch/quantization/observer.py:124: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:561: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
  if param.grad is not None:


Model Size: 0.00 MB
Saved Input and Output Image to drive/MyDrive/p2p-torch/


In [ ]:
for n, p in model.named_parameters():
    print(n, p.shape)

initial_down.0.weight torch.Size([64, 3, 4, 4])
initial_down.0.bias torch.Size([64])
down1.conv.0.weight torch.Size([128, 64, 4, 4])
down1.conv.1.weight torch.Size([128])
down1.conv.1.bias torch.Size([128])
down2.conv.0.weight torch.Size([256, 128, 4, 4])
down2.conv.1.weight torch.Size([256])
down2.conv.1.bias torch.Size([256])
down3.conv.0.weight torch.Size([512, 256, 4, 4])
down3.conv.1.weight torch.Size([512])
down3.conv.1.bias torch.Size([512])
down4.conv.0.weight torch.Size([512, 512, 4, 4])
down4.conv.1.weight torch.Size([512])
down4.conv.1.bias torch.Size([512])
down5.conv.0.weight torch.Size([512, 512, 4, 4])
down5.conv.1.weight torch.Size([512])
down5.conv.1.bias torch.Size([512])
down6.conv.0.weight torch.Size([512, 512, 4, 4])
down6.conv.1.weight torch.Size([512])
down6.conv.1.bias torch.Size([512])
bottleneck.0.weight torch.Size([512, 512, 4, 4])
bottleneck.0.bias torch.Size([512])
up1.conv.0.weight torch.Size([512, 512, 4, 4])
up1.conv.1.weight torch.Size([512])
up1.conv.1

### Trial 2

In [ ]:
import torch.quantization.quantize_fx as quantize_fx
import copy

model.eval()
model_to_quantize = copy.deepcopy(model)
model_to_quantize.eval()
qconfig_dict = {"": torch.quantization.default_dynamic_qconfig}
# prepare
model_prepared = quantize_fx.prepare_fx(model_to_quantize, qconfig_dict)
# no calibration needed when we only have dynamici/weight_only quantization
# quantize
model_prepared.eval()
model_quantized = quantize_fx.convert_fx(model_prepared)

# print_model_size(model_prepared)

/usr/local/lib/python3.7/dist-packages/torch/quantization/fx/quantization_patterns.py:444: UserWarning: dtype combination: (torch.float32, torch.qint8, torch.quint8) is not supported by Conv supported dtype combinations are: [(torch.quint8, torch.qint8, None)]
  "supported dtype combinations are: {}".format(dtypes, supported_dtypes))
/usr/local/lib/python3.7/dist-packages/torch/quantization/fx/quantization_patterns.py:969: UserWarning: dtype combination: (torch.float32, torch.qint8, torch.quint8) is not supported by <class 'torch.nn.modules.activation.LeakyReLU'> supported dtype combinations are: [(torch.quint8, torch.qint8, None)]
  "supported dtype combinations are: {}".format(dtypes, self.op, supported_dtypes[self.op]))
/usr/local/lib/python3.7/dist-packages/torch/quantization/fx/quantization_patterns.py:969: UserWarning: dtype combination: (torch.float32, torch.qint8, torch.quint8) is not supported by <class 'torch.nn.modules.conv.ConvTranspose2d'> supported dtype combinations are:

KeyError: ignored

In [ ]:
model_prepared.up1.dropout

Dropout(p=0.5, inplace=False)

### Trial 3

In [ ]:
import copy

model_to_quantize = copy.deepcopy(model)
qconfig_dict = {"": torch.quantization.get_default_qat_qconfig('qnnpack')}
model_to_quantize.train()
# prepare
model_prepared = quantize_fx.prepare_qat_fx(model_to_quantize, qconfig_dict)
# training loop (not shown)
# quantize
model_quantized = quantize_fx.convert_fx(model_prepared)
print_model_size(model_quantized)

torch.save(model_quantized, BASE_PATH + "checkpoints/p2p_fx.ptl")

NameError: ignored

## Trace

In [ ]:
import torch
import torchvision
from torch.utils.mobile_optimizer import optimize_for_mobile

model_quantized.eval()
example = torch.rand(1, 3, 512, 512)
traced_script_module = torch.jit.trace(model_quantized, example)
traced_script_module_optimized = optimize_for_mobile(traced_script_module)
traced_script_module_optimized._save_for_lite_interpreter(BASE_PATH + "checkpoints/p2p_quantization_fx.ptl")

<string>:5: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
<string>:5: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!


In [ ]:
import torch
import torchvision
from torch.utils.mobile_optimizer import optimize_for_mobile

model.eval()
example = torch.rand(1, 3, 512, 512)
traced_script_module = torch.jit.trace(model, example)
traced_script_module_optimized = optimize_for_mobile(traced_script_module)
traced_script_module_optimized._save_for_lite_interpreter(BASE_PATH + "checkpoints/p2p_traced.pt")

In [ ]:
scripted_module = torch.jit.script(model)
model_speedrun(scripted_module)

Saved Input and Output Image to drive/MyDrive/p2p-torch/


In [ ]:
# print_model_size(quantized_model)
model_speedrun(quantized_model)


elapsed time (seconds): 0.8
Saved Input and Output Image to drive/MyDrive/p2p-torch/


## Experimental

### V1

In [ ]:
import torch.nn as nn
from torch.quantization.quantize_fx import prepare_fx, convert_fx
from torch.quantization import default_dynamic_qconfig, float_qparams_weight_only_qconfig

# Full docs for supported qconfig for floating point modules/ops can be found in docs for quantization (TODO: link)
# Full docs for qconfig_dict can be found in the documents of prepare_fx (TODO: link)
qconfig_dict = {
    "object_type": [
        (nn.Conv2d, float_qparams_weight_only_qconfig),
        (nn.BatchNorm2d, float_qparams_weight_only_qconfig),
        (nn.ConvTranspose2d, default_dynamic_qconfig),
        (nn.ReLU, float_qparams_weight_only_qconfig),
    ]
}
# Deepcopying the original model because quantization api changes the model inplace and we want
# to keep the original model for future comparison
model_to_quantize = copy.deepcopy(model)
prepared_model = prepare_fx(model_to_quantize, qconfig_dict)
quantized_model = convert_fx(prepared_model)

print_model_size(quantized_model)
# print("prepared model:", prepared_model)
# print("quantized model", quantized_model)

AttributeError: ignored

### V2 (SUCCESS)

In [ ]:
import torch
from torch.quantization import get_default_qconfig
# Note that this is temporary, we'll expose these functions to torch.quantization after official releasee
from torch.quantization.quantize_fx import prepare_fx, convert_fx

dataset = MapDataset(BASE_PATH + "cherries/train/")
data_loader = DataLoader(dataset, batch_size=16)

def calibrate(model, data_loader):
    model.eval()
    with torch.no_grad():
        for image, target in data_loader:
            model(image)

torch.backends.quantized.engine = "qnnpack"
model.eval()
qconfig = get_default_qconfig("qnnpack")
qconfig_dict = {"": qconfig}
prepared_model = prepare_fx(model, qconfig_dict)  # fuse modules and insert observers
calibrate(prepared_model, data_loader)  # run calibration on sample data
quantized_model = convert_fx(prepared_model)  # convert the calibrated model to a quantized model

In [ ]:
qconfig

QConfig(activation=functools.partial(<class 'torch.quantization.observer.HistogramObserver'>, reduce_range=False), weight=functools.partial(<class 'torch.quantization.observer.MinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_tensor_symmetric))

## Garb

In [ ]:
print_model_size(quantized_model)
model_speedrun(quantized_model)

Model Size: 54.49 MB

elapsed time (seconds): 1.3
Saved Input and Output Image to drive/MyDrive/p2p-torch/


In [ ]:
model_trace(quantized_model, 'pix2pix_small_v3')

<string>:5: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
<string>:5: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!


In [ ]:
scripted = torch.jit.script(quantized_model)
print_model_size(scripted)

Model Size: 0.06 MB


In [ ]:
torch.jit.save(scripted, BASE_PATH + 'checkpoints/pix2pix_small_v4.ptl')

In [ ]:
torch.save(quantized_model, BASE_PATH + 'checkpoints/pix2pix_quantized_v3.pt')

In [ ]:
print_model_size(scripted)
model_speedrun(scripted)

Model Size: 0.06 MB

elapsed time (seconds): 1.8
Saved Input and Output Image to drive/MyDrive/p2p-torch/


In [ ]:
generator_path = BASE_PATH + 'checkpoints/pix2pix_small_v3.pt'
model = torch.load(generator_path, map_location="cpu")
print_model_size(model_prototype)
model_speedrun(model_prototype)

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:604: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  " silence this warning)", UserWarning)


Model Size: 0.00 MB

elapsed time (seconds): 1.4
Saved Input and Output Image to drive/MyDrive/p2p-torch/


In [ ]:
generator_path = BASE_PATH + 'checkpoints/pix2pix_quantized_v3.pt'
model = torch.jit.load(generator_path, map_location="cpu")
print_model_size(model_prototype)
model_speedrun(model_prototype)

RuntimeError: ignored

In [ ]:
# quantized_model

In [ ]:
torch.save(quantized_model, BASE_PATH + 'checkpoints/pix2pix_quantized_v5.pt')

# Sandbox

In [ ]:
torch.jit.save(torch.jit.script(quantized_model), BASE_PATH + 'checkpoints/pix2pix_quantized_v7.pth')

In [ ]:
loaded_quantized_model = torch.jit.load(BASE_PATH + 'checkpoints/pix2pix_quantized_v7.pth')

In [ ]:
benchmark_model = torch.jit.script(torch.load(BASE_PATH + 'checkpoints/pix2pix_benchmark.pt'))

In [ ]:
optimized_scripted_module = optimize_for_mobile(benchmark_model)
optimized_scripted_module._save_for_lite_interpreter(BASE_PATH + 'checkpoints/pix2pix_benchmark.ptl')

In [ ]:
optimized_scripted_module = optimize_for_mobile(loaded_quantized_model)
optimized_scripted_module._save_for_lite_interpreter(BASE_PATH + 'checkpoints/pix2pix_quantized_v7.ptl')

In [ ]:
print_model_size(loaded_quantized_model)

Model Size: 0.06 MB


In [ ]:
model_speedrun(optimized_scripted_module)


elapsed time (seconds): 5.4
Saved Input and Output Image to drive/MyDrive/p2p-torch/


In [ ]:
import torch

loaded_quantized_model = torch.jit.load(BASE_PATH + 'checkpoints/pix2pix_quantized_v6.ptl')

In [12]:
model_speedrun(model)

AttributeError: ignored

# Load Model

In [9]:
model = torch.load(BASE_PATH + 'checkpoints/pix2pix_traced_josh.pt', map_location="cpu")

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:604: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  " silence this warning)", UserWarning)


In [21]:
def model_speedrun(M):
  dataset = MapDataset(BASE_PATH + "cherries/val/")
  loader = DataLoader(dataset, batch_size=1)
  for x, y in loader:
      save_image(x * 0.5 + 0.5, BASE_PATH + "in.png")
      s = time.time()
      x = M(x)
      elapsed = time.time() - s
      x = x * 0.5 + 0.5
      save_image(x, BASE_PATH + "out.png")
      print('''\nelapsed time (seconds): {0:.1f}'''.format(elapsed))
      print("Saved Input and Output Image to " + BASE_PATH)
      break

In [22]:
model_speedrun(model)


elapsed time (seconds): 1.0
Saved Input and Output Image to drive/MyDrive/p2p-torch/


In [19]:
model

{'optimizer': {'param_groups': [{'amsgrad': False,
    'betas': (0.5, 0.999),
    'eps': 1e-08,
    'lr': 0.0002,
    'params': [0,
     1,
     2,
     3,
     4,
     5,
     6,
     7,
     8,
     9,
     10,
     11,
     12,
     13,
     14,
     15,
     16,
     17,
     18,
     19,
     20,
     21,
     22,
     23,
     24,
     25,
     26,
     27,
     28,
     29,
     30,
     31,
     32,
     33,
     34,
     35,
     36,
     37,
     38,
     39,
     40,
     41,
     42,
     43,
     44],
    'weight_decay': 0}],
  'state': {0: {'exp_avg': tensor([[[[-1.2929e-02, -2.5931e-02, -3.4452e-02, -4.1236e-02],
              [ 2.3588e-02,  1.3982e-02,  7.7962e-03,  1.0084e-03],
              [ 5.2239e-02,  4.8187e-02,  4.3423e-02,  3.4445e-02],
              [ 9.0384e-02,  8.4943e-02,  8.3081e-02,  7.4098e-02]],
    
             [[ 1.6307e+00,  1.6212e+00,  1.6090e+00,  1.6034e+00],
              [ 1.6459e+00,  1.6395e+00,  1.6302e+00,  1.6254e+00],
              [ 1.